In [1]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable, Iterable
from copy import deepcopy

## I) Déplacer un Tensor sur GPU

In [18]:
# on peut dénombrer les GPU disponibles avec:
torch.cuda.device_count()

1

In [2]:
# Le 'device' sur lequel un tenseur est stocké peut être défini à la création
# l'index après 'cuda' correspond à l'index du GPU si plusieurs GPUs sont disponibles
print("on CPU:")
print(torch.rand(3, 3, device="cpu"))
print("on GPU:")
print(torch.rand(3, 3, device="cuda:0"))

on CPU:
tensor([[0.9662, 0.6436, 0.8573],
        [0.1511, 0.5787, 0.7913],
        [0.3032, 0.5021, 0.9905]])
on GPU:
tensor([[0.6923, 0.9728, 0.2130],
        [0.0356, 0.0720, 0.6024],
        [0.0308, 0.4671, 0.6357]], device='cuda:0')


In [3]:
# on peut copier un tenseur sur un nouveau 'device' avec la méthode 'to'
print("before copying on GPU:")
t = torch.rand(3, 3)
print(t.device)
print("after copying on GPU:")
moved = t.to("cuda:0")
print(t.device, "\t", moved.device)

before copying on GPU:
cpu
after copying on GPU:
cpu 	 cuda:0


In [4]:
# il existe des méthode raccourcis
print(torch.rand(3, 3, device="cpu").cuda().device)
print(torch.rand(3, 3, device="cuda:0").cpu().device)

cuda:0
cpu


In [5]:
# on ne peut pas faire une opératio entre deux tensuers sur des 'device' différents
try:
    t * moved
except Exception as e:
    print(str(e))

Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!


## II) Déplacer un Module sur GPU

In [6]:
# on peut définir le 'device' des Modules built-in de pytorch
module = torch.nn.Linear(3, 5, device="cpu")

In [7]:
# il n'dxiste cependant pas d'attribut 'device'
try:
    module.device
except Exception as e:
    print(str(e))

'Linear' object has no attribute 'device'


In [8]:
# les objets Module ont aussi une méthode 'to', qui modifie l'objet 'inplace'
moved = module.to("cuda:0")
print(moved is module)

True


In [9]:
# moving a module moves all it's parameters
for p in module.parameters():
    print(p.device)

cuda:0
cuda:0


## III) Les convolutions

In [10]:
# La couche de convolution 2D est définie dans le namespace 'torch.nn'
conv = torch.nn.Conv2d(in_channels=3, out_channels=5, kernel_size=(3, 3), stride=(1, 1))
print(conv)

Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1))


In [11]:
# par défaut la convolution est appliquée sans padding, ce qui réduit la taille des featuremaps de sortie
conv(torch.rand(10, 3, 256, 256)).shape

torch.Size([10, 5, 254, 254])

In [14]:
# Le kwarg padding="same" permet de maintenir la taille de la feature map.
conv = torch.nn.Conv2d(in_channels=3, out_channels=5, kernel_size=(3, 3), stride=(1, 1), padding="same")
print(conv(torch.rand(10, 3, 256, 256)).shape)

torch.Size([10, 5, 256, 256])


## Exercice I

Définir et entraîner un modèle de classification d'images

In [ ]:
x = np.load("../datasets/CIFAR10/CIFAR-10.npy")
y = np.load("../datasets/CIFAR10/CIFAR-10-labels.npy")
labels = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

In [ ]:
def split(*args: tuple[np.ndarray], fracs: tuple[float] = (0.7, 0.15, 0.15)) -> list[tuple[np.ndarray]]:
    """
    split data in 
    """
    total = sum(fracs)
    fracs = tuple(f/total for f in fracs)
    L = len(args[0])
    indexes = np.random.permutation(L)
    start = 0
    results = []
    for f in fracs:
        end = start + int(f*L)
        results.append(tuple(array[indexes[start:end]] for array in args))
    return results



In [ ]:
train_data, val_data, test_data = split(x, y)

In [ ]:
x_test, y_test = test_data
f, axes = plt.subplots(figsize=[10, 10], nrows=3, ncols=3)
i = 0
for axs in axes:
    for ax in axs:
        ax.imshow(x_test[i].transpose(1, 2, 0))
        ax.axis("off")
        ax.set_title(labels[y_test[i]])
        i += 1

In [ ]:
def accuracy(predicted: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """
    returns the accuracy metric of a prediction
    """
    assert predicted.shape == target.shape
    return torch.sum(predicted == target).detach().cpu().item()/len(target)

def inputs_to_tensor(x) -> torch.Tensor:
    """
    converts an image array input into tensor
    """
    return torch.tensor(x/255., dtype=torch.float32)

def target_to_tensors(y: np.ndarray) -> torch.Tensor:
    """
    converts a target numpy array to tensor
    """
    return torch.tensor(y, dtype=torch.long)

def arrays_to_tensors(x: np.ndarray, y: np.ndarray) -> tuple[torch.Tensor]:
    """
    transforms numpy arrays to tensors
    """
    return (inputs_to_tensor(x), target_to_tensors(y))

class batchifyer:
    """
    transforms numpy arrays into batches of torch tensor
    """

    def __init__(self, *args: tuple[np.ndarray], n_batches: int = 1, batch_size: int = 100):
        self.args = args
        self.n_batches = n_batches
        self.batch_size = batch_size

    def __iter__(self):
        return self.get_batches()

    def get_batches(self) -> Iterable[torch.Tensor]:
        L = len(self.args[0])
        indexes = np.random.permutation(L)
        for i in range(self.n_batches):
            x, y = (array[indexes[i*self.batch_size:(i+1)*self.batch_size]] for array in self.args)
            yield arrays_to_tensors(x, y)

In [ ]:
class Block(torch.nn.Module):
    """
    A block is a set of convolution/max pooling/activation/batch normalization
    """

    def __init__(self, in_features: int, out_features: int, activation: Callable = torch.relu, kernel_size: tuple[int] = (3, 3), pool: tuple[int, int] = (2, 2)):
        super().__init__()
        self.convolution = torch.nn.Conv2d(in_features, out_features, kernel_size, (1, 1), padding="same")
        self.pool = torch.nn.MaxPool2d(pool)
        self.activation = activation
        self.batch_norm = torch.nn.BatchNorm2d(out_features)
    
    def forward(self, X):
        return self.batch_norm(self.activation(self.pool(self.convolution(X))))


class ImageClassifier(torch.nn.Module):
    """
    An Image classifier is a CNN that classifies Images
    """

    def __init__(self, in_features: int, n_classes: int, features: list[int], activation: Callable = torch.relu, kernel_size: tuple[int] = (3, 3), pool: tuple[int, int] = 2):
        super().__init__()
        self.blocks = torch.nn.ModuleList()
        for out_features in features:
            self.blocks.append(Block(in_features, out_features, activation, kernel_size, pool))
            in_features = out_features
        self.output = torch.nn.Conv2d(out_features, n_classes, kernel_size=(1, 1), padding="same")
    
    def forward(self, X):
        """
        Parameters
        ----------
        X : torch.Tensor
            tensor of floats of shape (N, C, H, W)
        """
        X = X.to(self.device)
        for block in self.blocks:
            X = block(X)
        X = self.output(X)
        N, C, H, W = X.shape
        return X.reshape(N, C, -1).mean(dim=-1)

    def predict(self, X: torch.Tensor) -> torch.Tensor:
        """
        perform a prediction of the model
        """
        self.eval()
        with torch.no_grad():
            Y = self(X)
        return Y.max(dim=1).indices
    
    def loss(self, X: torch.Tensor, Y: torch.Tensor) -> torch.Tensor:
        """
        loss of the model
        """
        y_pred = self(X)
        return F.cross_entropy(y_pred, Y.to(y_pred.device))
    
    def metric(self, X: torch.Tensor, Y: torch.Tensor):
        """
        metric of the model
        """
        y_pred = self.predict(X)
        return accuracy(y_pred, Y.to(y_pred.device))

    @property
    def device(self) -> torch.device:
        return self.output.weight.device

In [ ]:
def train_loop(model: torch.nn.Module, optimizer: torch.optim.Optimizer, train_data: Iterable[tuple[torch.Tensor]], val_data: Iterable[tuple[torch.Tensor]], n_steps: int = 1000, patience: int = 100, keep_best: bool = True):
    """
    train the model for the specified number of steps, or untilearly stopping
    """
    best_state = deepcopy(model.state_dict())
    best_step = 0
    best_metric = 0.
    try:
        for step in range(n_steps):
            optimizer.zero_grad()
            # train loss
            model.train()
            losses = []
            for x, y in train_data:
                loss = model.loss(x, y)
                loss.backward()
                losses.append(loss.item())
            loss = sum(losses)/len(losses)
            # val metric
            model.eval()
            metrics = []
            for x, y in val_data:
                metrics.append(model.metric(x, y))
            metric = sum(metrics) / len(metrics)
            # checkpointing
            if metric > best_metric:
                best_metric = metric
                best_step = step
                if keep_best:
                    best_state = deepcopy(model.state_dict())
            elif step - best_step > patience:
                print("early stoping")
                break
            # optimizer steping
            optimizer.step()
            # printing
            print(f"Step {step}: loss = {loss:.3g} metric = {metric:.2%}")

    except KeyboardInterrupt:
        print("interrupted by user")
    if keep_best:
        model.load_state_dict(best_state)

In [ ]:
model = ImageClassifier(3, len(labels), [32, 64, 128, 256])
model.to("cuda:0")
train = batchifyer(*train_data, n_batches=1, batch_size=10000)
val = batchifyer(*val_data, n_batches=1, batch_size=10000)
optimizer = torch.optim.Adam(model.parameters(), lr=1.0E-3)
train_loop(model, optimizer, train, val, n_steps=10000, patience=1000)

In [ ]:
x, y = arrays_to_tensors(*test_data)
y_pred = model.predict(x)
acc = accuracy(y_pred, y.to(y_pred.device))
print(f"accuracy {acc:.2%}")

In [ ]:
x_test, y_test = test_data
f, axes = plt.subplots(figsize=[10, 10], nrows=3, ncols=3)
i = 0
for axs in axes:
    for ax in axs:
        ax.imshow(x_test[i].transpose(1, 2, 0))
        ax.axis("off")
        ax.set_title(labels[y_pred[i]], color="g" if y_pred[i] == y_test[i] else "r")
        i += 1

## Exercice II

Définir et entraîner un modèle de segmentation d'images

In [ ]:
import PIL
from PIL import Image, ImageDraw

In [ ]:
def generate_polygons(image_shape: tuple[int, int], delta: int = 10, n_max_vertices: int = 5):
    """
    generate a set of random polygon parametrized as a list of (x, y) tuples

    Parameters
    ----------
    image_shape : tuple of int
        the (height, width) in pixels of the image to fill with polygons
    delta : int
        the 
    """
    assert n_max_vertices >= 3
    x_offset, y_offset = np.random.uniform(-delta, delta, 2)  # offset of the polygons grid
    x_spacing, y_spacing = np.arange(-delta, image_shape[1]+delta, delta)+x_offset, np.arange(-delta, image_shape[0]+delta, delta)+y_offset
    x_centers, y_centers = np.meshgrid(x_spacing, y_spacing, indexing="xy")  # coordinates of polygons centers
    Lx, Ly = x_centers.shape
    x_centers += np.random.uniform(-delta/2, delta/2, (Lx, Ly))
    y_centers += np.random.uniform(-delta/2, delta/2, (Lx, Ly))
    thetas = np.cumsum(np.random.uniform(1., 3., (Lx, Ly, n_max_vertices+1)), axis=-1)  # angle of each vertex of each polygon
    n_vertices = np.random.randint(3, n_max_vertices+1, (Lx, Ly, 1))  # number of vertices in each polygon
    thetas = thetas[..., :-1] * 2*np.pi / np.take_along_axis(thetas, n_vertices, axis=-1)
    mu, sigma = delta/2, delta/6
    radius = np.clip(np.random.normal(mu, sigma, (Lx, Ly, n_max_vertices)), 0, delta/2)  # radius of each vertex of eahc polygon
    return [[(x_center + r*np.cos(theta), y_center + r*np.sin(theta)) for r, theta, _ in zip(points_r, points_theta, range(point_n_vertices))]
            for points_r, points_theta, x_center, y_center, point_n_vertices
            in zip(radius.reshape(-1, n_max_vertices), thetas.reshape(-1, n_max_vertices), x_centers.reshape(-1), y_centers.reshape(-1), n_vertices.reshape(-1))]

def generate_image(width: int = 64, height: int = 64, delta: int = 20):
    """
    generate a an image of a polygon and it's target interior
    """
    shape = (height, width)
    img = Image.new("L", shape)
    target = Image.new("1", shape)
    img_draw = ImageDraw.Draw(img)
    target_draw = ImageDraw.Draw(target)
    n_polygons = np.random.randint(3, 6)
    for polygon in generate_polygons(shape, delta):
        img_draw.polygon(polygon, fill=None, outline="#ffffff")
        target_draw.polygon(polygon, fill="#ffffff", outline="#ffffff")
    return np.array(img, dtype=np.uint8), np.array(target, dtype=bool)

In [ ]:
x, y = generate_image(64, 64, 20)
f, axes = plt.subplots(figsize=[10, 5], ncols=2)
for ax, img in zip(axes, [x, y]):
    ax.imshow(img, cmap="viridis")
    ax.axis("off")

In [ ]:
class BatchesGenerator:
    """
    generate a batch of 'n' (input, target) observations when iterated over
    """
    def __init__(self, n_batches: int, batch_size : int):
        self.n_batches = n_batches
        self.batch_size = batch_size

    def __iter__(self):
        return self.get_batches()
    
    def get_batches(self):
        for _ in range(self.n_batches):
            inputs, targets = (np.stack(arrays, axis=0) for arrays in zip(*[generate_image() for _ in range(self.batch_size)]))
            yield torch.tensor(inputs/255., dtype=torch.float32), torch.tensor(targets, dtype=torch.bool)

In [ ]:
def F1_score(predicted: torch.Tensor, target: torch.Tensor) -> float:
    """
    returns the F1 score of a binary classification
    """
    assert predicted.shape == target.shape
    assert predicted.dtype == torch.bool
    assert target.dtype == torch.bool
    with torch.no_grad():
        tp = torch.sum(predicted & target).detach().cpu().item()
        fp = torch.sum(~predicted & target).detach().cpu().item()
        fn = torch.sum(predicted & ~target).detach().cpu().item()
    return (2*tp) / (2*tp + fp + fn)

In [ ]:
class ImageSegmenter(torch.nn.Module):

    def __init__(self, features: list[int], activation: Callable = torch.relu, kernel_size: tuple[int] = (3, 3), pool: tuple[int, int] = 2):
        super().__init__()
        self.encoder = torch.nn.ModuleList()
        in_features = 1
        for out_features in features:
            self.encoder.append(Block(in_features, out_features, activation, kernel_size, pool))
            in_features = out_features
        self.decoder = torch.nn.ModuleList()
        for out_features in features[-2::-1] + [1]:
            self.decoder.append(Block(in_features+out_features, out_features, activation, kernel_size, (1, 1)))
            in_features = out_features
        self.output = torch.nn.Conv2d(in_features, 1, (1, 1))
    
    def forward(self, X: torch.Tensor):
        feature_maps = []
        X = X.unsqueeze(1).to(self.device)
        for stage in self.encoder:
            feature_maps.append(X)
            X = stage(X)
        for stage in self.decoder:
            X = F.interpolate(X, scale_factor=2, mode="nearest")
            feature_map = feature_maps.pop()
            X = torch.cat([X, feature_map], dim=1)
            X = stage(X)
        self.output(X)
        return torch.sigmoid(X).squeeze(1)

    def predict(self, X: torch.Tensor) -> torch.Tensor:
        """
        """
        self.eval()
        with torch.no_grad():
            Y = self(X)
        return Y > 0.5
        
    def loss(self, X: torch.Tensor, Y: torch.Tensor) -> torch.Tensor:
        """
        """
        y_pred = self(X)
        return F.binary_cross_entropy(y_pred, (Y.float() / 255).to(y_pred.device))

    def metric(self, X: torch.Tensor, Y: torch.Tensor) -> torch.Tensor:
        """
        """
        y_pred = self.predict(X)
        return F1_score(y_pred > 0.5, Y.to(y_pred.device))

    @property
    def device(self) -> torch.device:
        return self.output.weight.device

In [ ]:
model = ImageSegmenter([8, 16, 32])
model.to("cuda:0")
generator = BatchesGenerator(1, 1000)
optimizer = torch.optim.Adam(model.parameters(), lr=1.0E-3)
train_loop(model, optimizer, generator, generator, n_steps=10000, patience=1000)

In [ ]:
x, y = next(iter(BatchesGenerator(1, 1)))
y_pred = model.predict(x)
f, axes = plt.subplots(figsize=[15, 5], ncols=3)
for ax, img in zip(axes, [x[0], y[0].cpu().numpy(), y_pred[0].cpu().numpy()]):
    ax.imshow(img, cmap="viridis")
    ax.axis("off")